# Hyper Parameter - Beispiel Regularization

Code-Beispiel für das Konzept von Hyper-Parametern am Beispiel der Regularisierungsstärke (`alpha`) im `Ridge` Modell.
Zudem wird die Hyper-Parameter Suche mittels `GridSearchCV` und `RandomizedSearchCV` am Beispiel gezeigt.

Die Hyper-Parameter Suche ist ein generelles Konzept. Wir werden noch komplexere Modelle mit mehreren Hyper-Parametern im Kurs kennenlernen. `GridSearchCV` und `RandomizedSearchCV` können analog verwendet werden.

## Setup

In [34]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_predict
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from scipy.stats import uniform

In [35]:
df = pd.read_csv('data/fish.csv')[['Width', 'Weight']].rename(columns={
    'Width': 'width (cm)',
    'Weight': 'weight (g)'
})
X=df[['width (cm)']]
y=df['weight (g)']

## Mehrere Parameter selbst ausprobieren

Hier machen wir die `GridSearch` selbst. In Praxis verwendet man lieber `GridSearchCV`, anstatt es selbst zu programmieren.
Der Code hier soll einzig helfen besser zu verstehen, was `GridSearchCV` genau macht.

Wir nutzen hier `sklearn.linear_model.Ridge`. `Ridge` ist die `Linear Regression mit L2-Regularisierung`.

In [36]:
model_01 = Pipeline([
    ('std', StandardScaler()),      # Für Regularisierung Daten vorher standard skalieren.
    ('ridge', Ridge(alpha=0.1))     # alpha heisst lambda in Folien.
])
y_hat = cross_val_predict(model_01, X, y)
print(f"alpha=0.1 => {mean_squared_error(y, y_hat)}")

model_1 = Pipeline([
    ('std', StandardScaler()),      # Für Regularisierung Daten vorher standard skalieren.
    ('ridge', Ridge(alpha=1.0))     # alpha heisst lambda in Folien.
])
y_hat = cross_val_predict(model_1, X, y)
print(f"alpha=1.0 => {mean_squared_error(y, y_hat)}")

model_10 = Pipeline([
    ('std', StandardScaler()),      # Für Regularisierung Daten vorher standard skalieren.
    ('ridge', Ridge(alpha=10.0))    # alpha heisst lambda in Folien.
])
y_hat = cross_val_predict(model_10, X, y)
print(f"alpha=10.0 => {mean_squared_error(y, y_hat)}")

alpha=0.1 => 34989.628195318706
alpha=1.0 => 34863.77036591203
alpha=10.0 => 34303.160119822445


Hier ist scheinbar eine starke Regularisierung (`alpha=10.0`) besser. Es wäre hier sinnvoll noch grössere alphas auszuprobieren.

## Mehrere Parameter mit `GridSearchCV` ausprobieren

`GridSearchCV` probiert alle gegebenen Parameter Kombinationen durch. Für jede Kombination führt es eine K-Fold-Cross-Validation durch und misst welche Kombination am besten funktioniert. Die beste Kombination von Parametern wird in `best_params_` gespeichert. Die Anzahl an Kombinationen wird bei vielen Hyperparametern schnell gross, meistens ist dann `RandomizedSearchCV` besser geeignet.

In [37]:
gs = GridSearchCV(
    Pipeline([
        ('std', StandardScaler()),  # Für Regularisierung Daten vorher standard skalieren.
        ('ridge', Ridge())          # alpha wird später von GridSearchCV gesetzt (param_grid)
    ]),
    param_grid={
        'ridge__alpha': [0.1, 1.0, 10]  # Probiere folgende Werte aus
    },
    scoring='neg_mean_squared_error'    # Wähle bestes Modell mit höchstem negativen MSE (bzw. tiefstem MSE).
)

gs.fit(X, y)

print(f"{gs.best_params_=}")

gs.best_params_={'ridge__alpha': 10}


## Mehrere Parameter mit `RandomizedSearchCV` ausprobieren

`RandomizedSearchCV` probiert `n_iter` (hier 10) zufällige Parameter Kombinationen durch. Für jede Kombination führt es eine K-Fold-Cross-Validation durch und misst welche Kombination am besten funktioniert. Die beste Kombination von Parametern wird in `best_params_` gespeichert.

In [38]:
rs = RandomizedSearchCV(
    Pipeline([
        ('std', StandardScaler()),  # Für Regularisierung Daten vorher standard skalieren.
        ('ridge', Ridge())          # alpha wird später von GridSearchCV gesetzt (param_grid)
    ]),
    param_distributions={
        'ridge__alpha': uniform(0.1, 10-0.1)    # Wähle zufälliges alpha gleich-verteilt (uniform) zwischen 0.1 und 10.0
    },
    n_iter=10,                                  # Trainiere 10 zufällige Modelle (mit zufälligen Parametern)
    scoring='neg_mean_squared_error',           # Wähle bestes Modell mit höchstem negativen MSE (bzw. tiefstem MSE).
    random_state=0                              # Wir fixieren hier den random_state, so gibt es immer das gleiche Resultat. Wenn man diese Zeile entfernt nehmen wir andere Zufallswerte.
)

rs.fit(X, y)

print(f"{rs.best_params_=}")

gs.best_params_={'ridge__alpha': 9.64026132896019}
